In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
file_list=[
    {
        "vehicle_name": "Chrysler Pacifica 2018",
        "dbc_name": "opendbc/chrysler_pacifica_2017_hybrid.dbc",
        "steer_brake_acc": [258, 284, 308],
        "file_name": "exported_csvs/6fb4948a7ebe670e_2019-11-12--00-35-53.bz2.csv"
    },
    {
        "vehicle_name": "Toyota Prius Prime 2020",
        "dbc_name": "opendbc/toyota_prius_2017_pt_generated.dbc",
        "steer_brake_acc": [37, 166, 581], # acc input looks strange
        "file_name": "exported_csvs/eb378db0873da597_2020-02-29--11-50-42--0--rlog.bz2.csv"
    },
    {
        "vehicle_name": "Acura ILX",
        "dbc_name": "opendbc/acura_ilx_2016_can_generated.dbc",
        "steer_brake_acc": [342, 420, 304], # acc input looks strange
        "file_name": "exported_csvs/rlog.bz2.csv"
    },    
]

In [5]:
file_path = "exported_csvs/rlog.bz2.csv"
sequences = list()
df = pd.read_csv(file_path, header=0)
values = df.values

In [6]:
len(values)

10882

In [7]:
targets = np.ones(len(values))*342

In [8]:
targets

array([342., 342., 342., ..., 342., 342., 342.])

In [12]:
df

,145,342,398,399,819,821,304,316,344,380,...,888,923,773,985,929,1034,1029,1064,1365,busTime
0,9234147259396915846,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,4285779475863
1,9234147259396915861,2.813075e+14,524387.0,1.583297e+14,60.0,5.131052e+11,1.055000e+03,1043.0,1.120986e+12,3.758113e+12,...,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,4285784902738
2,9234147259396915861,2.813075e+14,524402.0,1.583297e+14,60.0,5.131052e+11,1.055000e+03,1043.0,1.120986e+12,3.758113e+12,...,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,4285790760186
3,9234147259396915861,2.813075e+14,524402.0,1.638272e+14,60.0,5.131052e+11,1.070000e+03,1058.0,1.357210e+12,3.758113e+12,...,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,4285796476905
4,9234147259396915876,2.813075e+14,524357.0,1.550311e+14,60.0,5.131052e+11,1.085000e+03,1058.0,1.357210e+12,3.758113e+12,...,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,4285801909249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10877,9232951025206296631,2.812860e+14,524312.0,7.160020e+16,60.0,4.958246e+11,7.262130e+16,1073.0,8.755000e+03,3.990041e+12,...,1.284195e+18,16.0,1.099512e+14,32775.0,771751976.0,265.0,5.770237e+17,1558317.0,9.0,4345729928132
10878,9232951025189519371,2.812860e+14,524312.0,7.160130e+16,60.0,4.958246e+11,7.290278e+16,1028.0,8.710000e+03,3.990041e+12,...,1.284195e+18,16.0,1.099512e+14,32775.0,771751976.0,265.0,5.770237e+17,1558317.0,9.0,4345735204643
10879,9232951025189519371,2.812860e+14,524327.0,7.160130e+16,60.0,4.958246e+11,7.290278e+16,1028.0,8.710000e+03,3.990041e+12,...,1.284195e+18,16.0,1.099512e+14,32775.0,771751976.0,265.0,5.770237e+17,1558317.0,9.0,4345740926934
10880,9232951025189519371,2.812860e+14,524327.0,7.159360e+16,60.0,4.958246e+11,7.290278e+16,1043.0,8.725000e+03,3.998631e+12,...,1.284195e+18,47.0,1.099512e+14,32775.0,771751976.0,265.0,5.770237e+17,1558317.0,9.0,4345746924226


In [112]:
from random import randint

#This function creates the dataset for training the net
# It extracts snippets from the df dataframe above

def create_X_y(df, num_samples, sample_length):
    X = np.zeros([num_samples, sample_length])
    y = np.zeros(num_samples)
    for i in range(num_samples):
        # Randomly choose which signal to write from
        signal_index = randint(0,df.shape[1]-1)
        chunk = []
        #print(signal_index)
        starting_point = randint(0,df.shape[0]-sample_length)
        
        for j in range(sample_length):
            # Since signal is very long, only a snippet is taken
            X[i][j] = df.iat[j,signal_index]
            
        # Now create label based on what signal id we saved from    
        if df.columns[signal_index] == "342": #steering value for acura ilx
            np.append(y,1)
            y[i]=1
        else:
            y[i]=0
    return [X,y]
    

X, y = create_X_y(df, 1000, 100)


In [113]:
df.shape[1]

81

In [114]:
df.columns[1]

'342'

In [115]:
print(X.shape)

(1000, 100)


In [116]:
y.shape

(1000,)

In [9]:
from sklearn.model_selection import train_test_split


In [10]:
X_train, X_test, y_train, y_test = train_test_split(values, targets, test_size=0.2)

In [11]:
len(X_train)

8705

In [48]:
# FIXME: Prepare sequence somehow and adapt LSTM model to fit it
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation="sigmoid"))

NameError: name 'seq_len' is not defined